In [ ]:
!pip install --upgrade azureml-sdk[automl]
!pip install --upgrade azureml-sdk[explain]

In [ ]:
from azureml.core import Workspace, Datastore, Dataset

In [ ]:
ws = Workspace.from_config()

In [ ]:
dataStoreName = 'dataStoreName'
containerName = 'containerName'
accountName = 'storageAccountName'
accountKey = 'storageAccountKey'

if ws.datastores.get(dataStoreName):
    ds = ws.datastores.get(dataStoreName)
else:
    ds = Datastore.register_azure_blob_container(workspace=ws, 
                                             datastore_name=dataStoreName,
                                             container_name=containerName,
                                             account_name=accountName, 
                                             account_key=accountKey)

In [ ]:
# Create an in-memory Dataset on your local machine
datapath = ds.path('NameofYourFile.csv')
dataset = Dataset.auto_read_files(datapath)

In [ ]:
from azureml.core.runconfig import RunConfiguration

run_user_managed = RunConfiguration()
run_user_managed.environment.python.user_managed_dependencies = True

# Preparing the Azure Compute Target

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.runconfig import RunConfiguration
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Create a new runconfig object
run_temp_compute = RunConfiguration()

# Signal that you want to use AmlCompute to execute the script
run_temp_compute.target = "amlcompute"

# AmlCompute is created in the same region as your workspace
# Set the VM size for AmlCompute from the list of supported_vmsizes
run_temp_compute.amlcompute.vm_size = 'STANDARD_D2_V2'


# Choose a name for your CPU cluster
cpu_cluster_name = "ClusterNameGoesHere"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
datasetDefinition = dataset.get_definition()
features =  datasetDefinition.drop_columns('HasDetections')
labels =  datasetDefinition.keep_columns('HasDetections')

In [ ]:
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
import logging

automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             primary_metric = 'AUC_weighted',
                             iteration_timeout_minutes = 10,
                             iterations = 10,
                             verbosity = logging.INFO,
                             compute_target = cpu_cluster_name,
                             X = features,
                             y = labels,
                             n_cross_validations=3)

In [ ]:
from azureml.core.experiment import Experiment

experiment_name = 'ExperimentNameGoesHere'
experiment = Experiment(ws, experiment_name)

remote_run = experiment.submit(automl_config, show_output=True)

In [ ]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

In [ ]:
from sklearn.externals import joblib
from azureml.core.model import Model

modelNameFile = 'ModelFileNameGoesHere.pkl'
joblib.dump(value=fitted_model, filename=modelNameFile)

model = Model.register(model_path = modelNameFile,
                       model_name = "ModelNameGoesHere",
                       tags = {"key": "0.1"},
                       description = "test",
                       workspace = ws)